In [127]:
import pandas as pd
import numpy as np
import seaborn as sb

In [128]:
#Load in Combined data
data = pd.read_csv("/Users/carternorton/Desktop/FIB_UM_UU_Combined NoID.csv", index_col=0)

In [129]:
print(data.columns)

Index(['Diagnosis', 'Specific Diagnosis', 'NGS (1: F1, 2: Caris, 3: Tempus)',
       'Fsindel', 'TMB', 'MS MS (0: MSS, 1: MSI)', 'Age at Tx',
       'Sex (1:M, 2:F)',
       'Race (1: White, 2: Black, 3: Hispanic, 4: Asian, 5: others/unknown)',
       'Tx (1: Nivolumab, 2: Pembrolizumab, 3: Atezolizumab, 4: IPI + NIVO--> NIVO, 5: Ipi+Pembro)',
       'ECOG', '# of previous Sys Tx ', 'Date_of_First_Dose',
       'Date_Of_Last_Dose', '#ICI doses',
       '12th week response (0:CMR/CR, 1:PMR/PR, 2:CSD, 3:SMD/SD, 4:PMD/PD), CMR (complete metabolic response by PET))',
       'PFS (0:no, 1: yes)', 'PFS Date', 'OS (0: Alive, 1: Expired/Hospice)',
       '0S Date or last contact', 'Comments', 'Unnamed: 22', 'Unnamed: 23',
       'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27',
       'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31',
       'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35',
       'Unnamed: 36', 'Unnamed: 37', 'Comments.1'],
      dtype='object')


In [130]:
data["tx"] = data["Tx (1: Nivolumab, 2: Pembrolizumab, 3: Atezolizumab, 4: IPI + NIVO--> NIVO, 5: Ipi+Pembro)"]
print(data["tx"].value_counts(normalize=True))

2    0.542453
4    0.250000
1    0.183962
3    0.023585
Name: tx, dtype: float64


In [131]:
#Let's print a summary of "12th week response"
data["12_week_response"] = data["12th week response (0:CMR/CR, 1:PMR/PR, 2:CSD, 3:SMD/SD, 4:PMD/PD), CMR (complete metabolic response by PET))"]
print(data["12_week_response"].value_counts(normalize=True))
print(data["Fsindel"].value_counts(normalize=True))


4    0.476415
3    0.250000
1    0.146226
2    0.094340
0    0.033019
Name: 12_week_response, dtype: float64
0     0.547170
1     0.287736
2     0.080189
3     0.037736
18    0.009434
4     0.009434
15    0.004717
17    0.004717
26    0.004717
5     0.004717
21    0.004717
10    0.004717
Name: Fsindel, dtype: float64


In [132]:
### REFORMAT DATA ###
#Let's replace "0" and "1" with "response" in the 12_week_response column in data
data["12_week_response"] = data["12_week_response"].replace([0,1], "response")

#Let's replace "2", "3", and "4" with "no_response" in the 12_week_response column in data
data["12_week_response"] = data["12_week_response"].replace([2,3,4], "no_response")

#Let's replace all indel values > 0 with "1" in the Fsindel column in data
data["Fsindel"] = data["Fsindel"].replace([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,18,21,26], 1)

#Let's replace 

#Let's replace all tx values != 4 with "mono" and 4 with "dual" in the tx column in data
data["tx"] = data["tx"].replace([1,2,3,5], "mono")
data["tx"] = data["tx"].replace(4, "dual")

#Let's replace > 0 in '# of previous Sys Tx ' with "any" and 0 with "none" in the '# of previous Sys Tx ' column in data
data["full_previous_sys_tx"] = data["# of previous Sys Tx "]
data["# of previous Sys Tx "] = data["# of previous Sys Tx "].replace([1,2,3,4,5,6,7,8,9,10], "yes")
data["# of previous Sys Tx "] = data["# of previous Sys Tx "].replace(0, "no")

#If 'TMB' is less than 10, replace with TMB-L, otherwise replace with TMB-H
for patient in data.index:
    if data.loc[patient, "TMB"] == "<10":
        data.loc[patient, "TMB"] = "TMB-L"
    elif float(data.loc[patient, "TMB"]) >= 10:
        data.loc[patient, "TMB"] = "TMB-H"
    else:
        data.loc[patient, "TMB"] = "TMB-L"




In [205]:
### Generate Counts ###
print(data["# of previous Sys Tx "].value_counts())

yes    147
no      65
Name: # of previous Sys Tx , dtype: int64


In [170]:
### T Tests ###

#Let's convert 'dates' to datetime
data['Date_of_First_Dose'] = pd.to_datetime(data["Date_of_First_Dose"])
data['PFS Date'] = pd.to_datetime(data["PFS Date"])
data["0S Date or last contact"] = pd.to_datetime(data["0S Date or last contact"])
data["Treatment_Length"] = (data["PFS Date"] - data["Date_of_First_Dose"]).dt.days / 30.4
data["OS_Length"] = (data["0S Date or last contact"] - data["Date_of_First_Dose"]).dt.days / 30.4

#Let's do t test of treatment length and Fsindel
from scipy.stats import ttest_ind
stat, p = (ttest_ind( data[data["Fsindel"] == 0]["Treatment_Length"], data[data["Fsindel"] == 1]["Treatment_Length"], equal_var=False))
p = round(p,4)
print("P value between Fsindel -+ PFS time is: ", p)
print("median PFS length for FS-A: ", np.median(data[data["Fsindel"] == 0]["Treatment_Length"]))
print("median PFS length for FS-P: ", np.median(data[data["Fsindel"] == 1]["Treatment_Length"]))

#Let's do t test of OS length and Fsindel
stat, p = (ttest_ind( data[data["Fsindel"] == 0]["OS_Length"], data[data["Fsindel"] == 1]["OS_Length"], equal_var=False))
p = round(p,4)
print("P value between Fsindel -+ OS time is: ", p)
print("median OS length for FS-A: ", np.median(data[data["Fsindel"] == 0]["OS_Length"]))
print("median OS length for FS-P: ", np.median(data[data["Fsindel"] == 1]["OS_Length"]))

#Let's do t test of treatment length and mutation burden 
stat, p = (ttest_ind(data[data["TMB"]=="TMB-L"]["Treatment_Length"], data[data["TMB"]=="TMB-H"]["Treatment_Length"], equal_var=False))
p = round(p,4)
print("P value between TMB L/H PFS time is: ", p)
print("median PFS length for TMB-L: ", np.median(data[data["TMB"]=="TMB-L"]["Treatment_Length"]))
print("median PFS length for TMB-H: ", np.median(data[data["TMB"]=="TMB-H"]["Treatment_Length"]))

#Let's do t test of OS treatment length and mutation burden
stat, p = (ttest_ind(data[data["TMB"]=="TMB-L"]["OS_Length"], data[data["TMB"]=="TMB-H"]["OS_Length"], equal_var=False))
p = round(p,4)
print("P value between TMB L/H OS time is: ", p)
print("median OS length for TMB-L: ", np.median(data[data["TMB"]=="TMB-L"]["OS_Length"]))
print("median OS length for TMB-H: ", np.median(data[data["TMB"]=="TMB-H"]["OS_Length"]))


#Let's do t test of treatment length and mono/dual tx
stat, p = (ttest_ind(data[data["tx"]=="mono"]["Treatment_Length"], data[data["tx"]=="dual"]["Treatment_Length"], equal_var=False))
p = round(p,4)
print("P value between mono/dual tx PFS time is: ", p)
print("median PFS length for mono tx is: ", np.median(data[data["tx"]=="mono"]["Treatment_Length"]))
print("median PFS length for dual tx is:", np.median(data[data["tx"]=="dual"]["Treatment_Length"]))

#Let's do t test of OS treatment length and mono/dual tx
stat, p = (ttest_ind(data[data["tx"]=="mono"]["OS_Length"], data[data["tx"]=="dual"]["OS_Length"], equal_var=False))
p = round(p,4)
print("P value between mono/dual tx OS time is: ", p)
print("median OS length for mono tx is: ", np.median(data[data["tx"]=="mono"]["OS_Length"]))
print("median OS length for dual tx is:", np.median(data[data["tx"]=="dual"]["OS_Length"]))

#Let's t test of PFS and previous sys tx
stat, p = (ttest_ind(data[data["# of previous Sys Tx "]=="yes"]["Treatment_Length"], data[data["# of previous Sys Tx "]=="no"]["Treatment_Length"], equal_var=False))
p = round(p,4)
print("P value between previous sys tx PFS time is: ", p)
print("median PFS length for no previous tx is: ", np.median(data[data["# of previous Sys Tx "]=="no"]["Treatment_Length"]))
print("median PFS length for previous tx is:, ", np.median(data[data["# of previous Sys Tx "]=="yes"]["Treatment_Length"]))

#Let's t test of OS and previous sys tx
stat, p = (ttest_ind(data[data["# of previous Sys Tx "]=="yes"]["OS_Length"], data[data["# of previous Sys Tx "]=="no"]["OS_Length"], equal_var=False))
p = round(p,4)
print("P value between previous sys tx OS time is: ", p)
print("median OS length for no previous tx is: ", np.median(data[data["# of previous Sys Tx "]=="no"]["OS_Length"]))
print("median OS length for previous tx is:, ", np.median(data[data["# of previous Sys Tx "]=="yes"]["OS_Length"]))







P value between Fsindel -+ PFS time is:  0.4724
median PFS length for FS-A:  4.029605263157896
median PFS length for FS-P:  4.802631578947369
P value between Fsindel -+ OS time is:  0.7261
median OS length for FS-A:  10.263157894736842
median OS length for FS-P:  9.983552631578949
P value between TMB L/H PFS time is:  0.0015
median PFS length for TMB-L:  3.4539473684210527
median PFS length for TMB-H:  7.960526315789474
P value between TMB L/H OS time is:  0.2851
median OS length for TMB-L:  9.342105263157896
median OS length for TMB-H:  11.513157894736842
P value between mono/dual tx PFS time is:  0.1025
median PFS length for mono tx is:  4.111842105263158
median PFS length for dual tx is: 5.921052631578948
P value between mono/dual tx OS time is:  0.9566
median OS length for mono tx is:  10.328947368421053
median OS length for dual tx is: 9.046052631578949
P value between previous sys tx PFS time is:  0.0003
median PFS length for no previous tx is:  9.210526315789474
median PFS lengt

In [167]:
### Chi squared tests ###

#Let's perform a chi-squared test to see if there is a significant difference in response between the tx groups
from scipy.stats import chi2_contingency
from scipy.stats import chi2

# Response
table = pd.crosstab(data["12_week_response"], data["tx"])
stat, p, dof, expected = chi2_contingency(table)
print("The p value of the chi square of response and mono/dual tx is: " + str(round(p,3)))

#Print response rate for mono and dual tx
print("Response rate for mono tx is: " + str(round(100*data[data["tx"]=="mono"]["12_week_response"].value_counts(normalize=True)[1],1)) + "%")
print("Response rate for dual tx is: " + str(round(100*data[data["tx"]=="dual"]["12_week_response"].value_counts(normalize=True)[1],1)) + "%")


# Previous systemic treatments and response
table = pd.crosstab(data["12_week_response"], data["# of previous Sys Tx "])
stat, p, dof, expected = chi2_contingency(table)
print("The p value of the chi square of response rate and previous systemic treatment is: " + str(round(p,3)))

#Print response rate for previous systemic treatment
print("Response rate for no previous systemic treatment is: " + str(round(100*data[data["# of previous Sys Tx "]=="no"]["12_week_response"].value_counts(normalize=True)[1],1)) + "%")
print("Response rate for previous systemic treatment is: " + str(round(100*data[data["# of previous Sys Tx "]=="yes"]["12_week_response"].value_counts(normalize=True)[1],1)) + "%")

#Mutation burden and response
table = pd.crosstab(data["12_week_response"], data["TMB"])
stat, p, dof, expected = chi2_contingency(table)
print("The p value of the chi square of response rate and mutation burden is: " + str(round(p,6)))

#Print response rate for mutation burden
print("Response rate for low mutation burden is: " + str(round(100*data[data["TMB"]=="TMB-L"]["12_week_response"].value_counts(normalize=True)[1],1)) + "%")
print("Response rate for high mutation burden is: " + str(round(100*data[data["TMB"]=="TMB-H"]["12_week_response"].value_counts(normalize=True)[1],1)) + "%")

#Fsi and response
table = pd.crosstab(data["12_week_response"], data["Fsindel"])
stat, p, dof, expected = chi2_contingency(table)

print("The p value of the chi square of response rate and Fsi is: " + str(round(p,3)))

#Print response rate for Fsi
print("Response rate for no Fsi is: " + str(round(100*data[data["Fsindel"]==0]["12_week_response"].value_counts(normalize=True)[1],1)) + "%")
print("Response rate for Fsi is: " + str(round(100*data[data["Fsindel"]==1]["12_week_response"].value_counts(normalize=True)[1],1)) + "%")





The p value of the chi square of response and mono/dual tx is: 0.215
Response rate for mono tx is: 15.7%
Response rate for dual tx is: 24.5%
The p value of the chi square of response rate and previous systemic treatment is: 0.135
Response rate for no previous systemic treatment is: 24.6%
Response rate for previous systemic treatment is: 15.0%
The p value of the chi square of response rate and mutation burden is: 0.000166
Response rate for low mutation burden is: 10.2%
Response rate for high mutation burden is: 32.0%
The p value of the chi square of response rate and Fsi is: 0.057
Response rate for Fsi is: 24.0%
Response rate for no Fsi is: 12.9%


In [181]:
### ANOVA Analyses ###
import scipy.stats as stats
from scipy.stats import f_oneway

#ICI Doses
statistic, p_value = stats.f_oneway(data[(data["Fsindel"]==0) & (data["TMB"] == "TMB-L")]["#ICI doses"], data[(data["Fsindel"]==0) & (data["TMB"] == "TMB-H")]["#ICI doses"], data[(data["Fsindel"]==1) & (data["TMB"] == "TMB-L")]["#ICI doses"], data[(data["Fsindel"]==1) & (data["TMB"] == "TMB-H")]["#ICI doses"])

print("ANOVA for ICI Doses:")
print("F-statistic:", round(statistic, 3))
print("p-value:", round(p_value, 3))

#Age at diagnosis
statistic, p_value = stats.f_oneway(data[(data["Fsindel"]==0) & (data["TMB"] == "TMB-L")]["Age at Tx"], data[(data["Fsindel"]==0) & (data["TMB"] == "TMB-H")]["Age at Tx"], data[(data["Fsindel"]==1) & (data["TMB"] == "TMB-L")]["Age at Tx"], data[(data["Fsindel"]==1) & (data["TMB"] == "TMB-H")]["Age at Tx"])

print("ANOVA for Age at diagnosis:")
print("F-statistic:", round(statistic, 3))
print("p-value:", round(p_value, 3))

#ECOG
#Remove the one patient with NA for ECOG
data_1 = data[data["ECOG"] >= 0]
statistic, p_value = stats.f_oneway(data_1[(data_1["Fsindel"]==0) & (data_1["TMB"] == "TMB-L")]["ECOG"], data_1[(data_1["Fsindel"]==0) & (data_1["TMB"] == "TMB-H")]["ECOG"], data_1[(data_1["Fsindel"]==1) & (data_1["TMB"] == "TMB-L")]["ECOG"], data_1[(data_1["Fsindel"]==1) & (data_1["TMB"] == "TMB-H")]["ECOG"])

print("ANOVA for ECOG:")
print("F-statistic:", round(statistic, 3))
print("p-value:", round(p_value, 3))








ANOVA for ICI Doses:
F-statistic: 2.622
p-value: 0.052
ANOVA for Age at diagnosis:
F-statistic: 5.393
p-value: 0.001
ANOVA for ECOG:
F-statistic: 1.365
p-value: 0.254


In [206]:
### Large Chi Squared Analysis ###

#Let's see if race 'Race (1: White, 2: Black, 3: Hispanic, 4: Asian, 5: others/unknown)' and genomic group (TMB/Fsindel) are independent

#First, let's create a new column that combines TMB and Fsindel
data["TMB_Fsindel"] = data["TMB"] + "_" + data["Fsindel"].astype(str)

#Race and genomic group
#Now, let's create a contingency table
c_table = pd.crosstab(index=data["Race (1: White, 2: Black, 3: Hispanic, 4: Asian, 5: others/unknown)"],columns=data["TMB_Fsindel"])

# Perform the chi-square test
chi2_stat, p_value, dof, expected = stats.chi2_contingency(c_table)

# Print the results
print("Chi-square for Race:")
print("p-value:", round(p_value,3))

#Previous tx and genomic group
#Now, let's create a contingency table
c_table = pd.crosstab(index=data["# of previous Sys Tx "],columns=data["TMB_Fsindel"])

# Perform the chi-square test
chi2_stat, p_value, dof, expected = stats.chi2_contingency(c_table)

# Print the results
print("Chi-square for Previous Tx:")
print("p-value:", round(p_value,3))

#Microsatellite status and genomic group
#Now, let's create a contingency table
c_table = pd.crosstab(index=data['MS MS (0: MSS, 1: MSI)'],columns=data["TMB_Fsindel"])

# Perform the chi-square test
chi2_stat, p_value, dof, expected = stats.chi2_contingency(c_table)

# Print the results
print("Chi-square for Microsatellite status:")
print("p-value:", p_value)
print("All of these are in the TMB-H, FS-P")

#Disease site and genomic group
#Now, let's create a contingency table
c_table = pd.crosstab(index=data['Specific Diagnosis'],columns=data["TMB_Fsindel"])

# Perform the chi-square test
chi2_stat, p_value, dof, expected = stats.chi2_contingency(c_table)

# Print the results
print("Chi-square for Diagnosis:")
print("p-value:", round(p_value,3))

#Gender and genomic group
#Now, let's create a contingency table
c_table = pd.crosstab(index=data['Sex (1:M, 2:F)'],columns=data["TMB_Fsindel"])

# Perform the chi-square test
chi2_stat, p_value, dof, expected = stats.chi2_contingency(c_table)

# Print the results
print("Chi-square for Gender:")
print("p-value:", round(p_value,3))





Chi-square for Race:
p-value: 0.775
Chi-square for Previous Tx:
p-value: 0.274
Chi-square for Microsatellite status:
p-value: 7.818813228498543e-11
All of these are in the TMB-H, FS-P
Chi-square for Diagnosis:
p-value: 0.021
Chi-square for Gender:
p-value: 0.516
